## Módulos

In [123]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from category_encoders import MEstimateEncoder

## Carregar dados

In [124]:
X_full = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv', index_col='Id')

In [125]:
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

In [126]:
print(f'X_test_full data shape: {X_test_full.shape}')
print(f'y data shape: {y.shape}')
print(f'X_full data shape: {X_full.shape}')

X_test_full data shape: (1459, 79)
y data shape: (1460,)
X_full data shape: (1460, 79)


## Train and Test Split

In [127]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, train_size=0.8, test_size=0.2, random_state=0)

## Seleção de colunas (categóricas e numéricas)

In [128]:
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

## Preprocessing Steps and Pipeline

In [209]:
numerical_transformer = SimpleImputer(strategy='mean')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model_mk2 = XGBRegressor(colsample_bytree=0.4, learning_rate=0.03, max_depth=4, n_estimators=7500)

# Bundle preprocessing and modeling code in a pipeline
pipeline_mk2 = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model_mk2)
                     ])

# Preprocessing of training data, fit model 
pipeline_mk2.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
mk2_preds = pipeline_mk2.predict(X_valid)

print('mk2 MAE:', mean_absolute_error(y_valid, mk2_preds))

mk2 MAE: 15375.093134631848


In [ ]:
# model_mk2 = XGBRegressor(colsample_bytree=0.4, learning_rate=0.03, max_depth=4, n_estimators=4500)
# mk2 MAE: 15488.651460830479

# model_mk2 = XGBRegressor(colsample_bytree=0.4, learning_rate=0.03, max_depth=4, n_estimators=5500)
# mk2 MAE: 15485.14960134846

# model_mk2 = XGBRegressor(colsample_bytree=0.4, learning_rate=0.03, max_depth=4, n_estimators=6500)
# mk2 MAE: 15473.784821810788

# model_mk2 = XGBRegressor(colsample_bytree=0.4, learning_rate=0.03, max_depth=4, n_estimators=5500)
# mk2 MAE: 15378.602244755994

# model_mk2 = XGBRegressor(colsample_bytree=0.4, learning_rate=0.03, max_depth=4, n_estimators=6500)
# mk2 MAE: 15375.135100064212

## Output

In [212]:
mk2_test_predictions = pipeline_mk2.predict(X_test)
output = pd.DataFrame({'Id': X_test_full.index,
                       'SalePrice': mk2_test_predictions})
output.to_csv('submission_mk2.csv', index=False)